In [ ]:
!uv pip install google-adk --prerelease=allow

In [ ]:
# Set Google AI Studio API key
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [ ]:
# Set constants
GEMINI_MODEL = "gemini-2.0-flash"
APP_NAME = "linkedin_content_creation_app"
USER_ID = "ashish_bamania"
SESSION_ID = "content_session_01"

In [ ]:
from google.adk.agents.llm_agent import LlmAgent
from google.adk.tools import google_search

# Google Research Agent
google_research_agent = LlmAgent(
    name = "Google_Research_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are a Research AI agent.
    Use Google search to find recent and relevant information on the provided topic.
    **Do not** add information on your own apart from what you found after the Google search.
    Summarize key points, statistics, and insights in bullet points.
    """,
    description = "Researches on a given topic using Google Search.",
    tools = [google_search],
    output_key = "google_research_summary"
)

In [ ]:
!uv pip install -qU duckduckgo-search langchain-community --prerelease=allow

In [ ]:
# Set up Duckduckgo search tool from Langchain

from google.adk.tools.langchain_tool import LangchainTool
from langchain_community.tools import DuckDuckGoSearchRun

# Instantiate the tool
duckduckgo_tool_instance = DuckDuckGoSearchRun(
    max_results = 5,
)

# Wrap the tool in the LangchainTool class from ADK
adk_duckduckgo_tool = LangchainTool(
    tool=duckduckgo_tool_instance,
)

In [ ]:
# DuckDuckGo Research Agent
duckduckgo_research_agent = LlmAgent(
    name = "DuckDuckGo_Research_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are a Research AI agent.
    Use DuckDuckGo search to find recent and relevant information on the provided topic.
    **Do not** add information on your own apart from what you found after the DuckDuckGo search.
    Summarize key points, statistics, and insights in bullet points.
    """,
    description = "Researches on a given topic using DuckDuckGo Search.",
    tools = [adk_duckduckgo_tool],
    output_key = "duckduckgo_research_summary"
)

In [ ]:
from google.adk.agents.parallel_agent import ParallelAgent

# Parallel Web Research Agent
parallel_web_research_agent = ParallelAgent(
    name = "Parallel_Web_Research_Agent",
    sub_agents = [google_research_agent, duckduckgo_research_agent]
)

In [ ]:
# Merge Research Agent (combine Google and DuckDuckGo results)
merge_research_agent = LlmAgent(
    name = "Merge_Research_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are a Content Merging AI.
    Merge the outputs provided by the Google and DuckDuckGo agents in the session state under the keys 'google_research_summary' and 'duckduckgo_research_summary', respectively.
    **Do not** add information on your own.
    Remove duplicates.
    Summarize key points clearly in bullet points.
    """,
    description = "Merges research results from Google and DuckDuckGo.",
    output_key = "merged_research_summary"
)

In [ ]:
# Writer Agent
writer_agent = LlmAgent(
    name = "Writer_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are a LinkedIn Content Writer AI.
    Based on the research summary provided in session state with the key 'merged_research_summary', draft a LinkedIn post that includes:
    - A compelling hook and heading
    - Key insights (describing the topic in technical detail if available)
    - A clear call-to-action

    Ensure the tone is professional and engaging.
    **Do not**  add links to any external resources.
    **Do not**  add your own information and your writing should be totally based on the research summary.
    """,
    description = "Drafts initial LinkedIn post based on research results.",
    output_key = "initial_draft"
)

In [ ]:
# SEO Optimizer Agent
seo_optimizer_agent = LlmAgent(
    name = "SEO_Optimizer_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are an SEO Optimizer AI.
    Enhance the draft provided in session state with the key 'initial_draft' by:
    - Incorporating relevant keywords
    - Adding appropriate hashtags (**only** 3)
    - Ensuring optimal length (not omiting relevant research details) and readability
    - Formatting for LinkedIn best practices
    """,
    description="Optimizes content for SEO and LinkedIn formatting.",
    output_key="seo_optimized_draft"
)

In [ ]:
# Final draft guardrail

import re
import copy
from typing import Optional
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse
from google.genai import types

def final_draft_guardrail(callback_context: CallbackContext, llm_response: LlmResponse) -> Optional[LlmResponse]:
    agent_name = callback_context.agent_name

    print(f"[Callback] Running guardrail for: {agent_name}")

    if not llm_response.content or not llm_response.content.parts:
        print("[Callback] No content found in LLM response.")
        return None

    part = llm_response.content.parts[0]

    if part.function_call or not part.text:
        print("[Callback] Skipping guardrail because response contains function call or no text.")
        return None

    original_text = part.text
    text = original_text
    modified = False

    # Remove markdown links like [text](url)
    new_text = re.sub(r'\[.*?\]\(.*?\)', '', text)

    if new_text != text:
        print("[Callback] Markdown links removed.")
        text = new_text
        modified = True

    # Remove raw URLs
    new_text = re.sub(r'https?://\S+|www\.\S+', '', text)

    if new_text != text:
        print("[Callback] Raw URLs removed.")
        text = new_text
        modified = True

    # Remove markdown formatting
    text_no_md = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text_no_md = re.sub(r'__(.*?)__', r'\1', text_no_md)
    text_no_md = re.sub(r'\*(.*?)\*', r'\1', text_no_md)
    text_no_md = re.sub(r'_(.*?)_', r'\1', text_no_md)

    if text_no_md != text:
        print("[Callback] Markdown formatting removed.")
        text = text_no_md
        modified = True

    # Remove leftover [ ] brackets
    if '[' in text or ']' in text:
        text = text.replace('[', '').replace(']', '')
        print("[Callback] Square brackets removed.")
        modified = True

    # Replace markdown bullets with dashes
    bullet_converted = re.sub(r'^\s*\*\s+', '- ', text, flags=re.MULTILINE)

    if bullet_converted != text:
        print("[Callback] Markdown bullet points replaced with dashes.")
        text = bullet_converted
        modified = True

    # Remove intro phrases
    intro_patterns = [
        r'^\s*Here is.*?:\s*',
        r'^\s*Final draft.*?:\s*',
        r'^\s*Here\'s.*?:\s*',
        r'^\s*Your post.*?:\s*'
    ]

    for pattern in intro_patterns:
        new_text = re.sub(pattern, '', text, flags=re.IGNORECASE)

        if new_text != text:
            print(f"[Callback] Intro phrase removed using pattern: {pattern}")
            text = new_text
            modified = True

    # Extract hashtags
    hashtags = re.findall(r"#\w+", text)

    if hashtags:
        print(f"[Callback] Extracted hashtags: {hashtags}")
        modified = True

    if len(hashtags) > 3:
        print(f"[Callback] More than 3 hashtags found. Trimming to: {hashtags[:3]}")
        hashtags = hashtags[:3]

    hashtags = [tag.lower() for tag in hashtags]

    # Remove hashtags from main text
    cleaned_text = re.sub(r"#\S+", "", text)

    if cleaned_text != text:
        print("[Callback] Hashtags removed from main text.")
        text = cleaned_text
        modified = True

    if not modified:
        print("[Callback] No modifications made to the response text.")
        return None

    # Create final text
    final_text = (
        f"{text.strip()}\n\n"
        f"💡 Like this perspective? Follow Dr. Ashish Bamania to stay connected: https://www.linkedin.com/in/ashishbamania "
        f"{' '.join(hashtags)}"
    )

    print("[Callback] Final CTA and hashtags added.")

    modified_parts = [copy.deepcopy(part)]
    modified_parts[0].text = final_text.strip()

    new_response = LlmResponse(
        content=types.Content(role="model", parts=modified_parts),
        grounding_metadata=llm_response.grounding_metadata
    )

    print("[Callback] Guardrail applied successfully. Modified text returned.")

    return new_response

In [ ]:
final_draft_writing_agent = LlmAgent(
    name = "Final_Draft_Writing_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are a LinkedIn Post Drafting AI.
    Review the optimized content provided in the session state with the key 'seo_optimized_draft' for:
    - Clarity and coherence
    - Grammatical accuracy
    - Consistent tone and style
    - Engagement and professionalism

    Remove Markdown formatting including any *.
    Make sure only the **best 3** hastags are used.

    **Add** appropriate emojis in the post.

    Provide the final version ready for posting.
    Output **only** the final LinkedIn-ready post without any introductory or explanatory text.
    **Do not** add links to any external resources.
    """,
    description = "Produces final LinkedIn post with guardrails.",
    output_key = "final_post",
    after_model_callback = final_draft_guardrail
)

In [ ]:
!uv pip install "crewai[tools]" --prerelease=allow

In [ ]:
# Set up DALL-E image creation tool from CrewAI

from google.adk.tools.crewai_tool import CrewaiTool
from crewai_tools import DallETool

dalle_tool_instance = DallETool(model="dall-e-3",
                       size="1024x1024",
                       quality="hd",
                       n=1)

adk_dalle_tool = CrewaiTool(
    name = "DALLE_Image_Creation_Tool",
    description="Generates images using DALLE.",
    tool=dalle_tool_instance,
)

In [ ]:
# Set up OpenAI API key

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
# Image Generation Agent
image_generation_agent = LlmAgent(
    name = "Image_Generation_Agent",
    model = GEMINI_MODEL,
    instruction = """
    You are an Image Generation AI.
    Generate an beautiful, minimalistic and attention grabbing image for the context provided in the session state with the key 'final_post'.
    It should visually represent the key insights of the final LinkedIn post.
    This image will be posted with this LinkedIn post.
    **Please avoid** using any text in the image.
    Return the URL of the infographic image in your response.
    """,
    description = "Generates image using DALL-E.",
    tools = [adk_dalle_tool],
    output_key = "image_url"
)

In [ ]:
from google.adk.agents.sequential_agent import SequentialAgent

linkedin_content_creation_pipeline = SequentialAgent(
    name="LinkedIn_Content_Creation_Pipeline",
    sub_agents=[parallel_web_research_agent, writer_agent, seo_optimizer_agent, final_draft_writing_agent, image_generation_agent]
)

In [ ]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

# Set up Session
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

# Set up Runner
runner = Runner(agent=linkedin_content_creation_pipeline, app_name=APP_NAME, session_service=session_service)

In [ ]:
from google.genai import types

def call_agent(query):
    '''
    Helper function to call the agent with a query.
    '''
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.is_final_response():
            author = event.author
            final_response = event.content.parts[0].text
            print(f"{author} Response:\n{final_response}\n")

In [ ]:
call_agent("how to master patience")